In [56]:
import openpyxl

In [57]:
def import_excel_data(file_path, sheet_name):
    # Load the workbook and select the desired sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[sheet_name]

    # Get labels from row 2 (B2 to G2)
    labels = [sheet.cell(row=2, column=col).value for col in range(2, 8)]  # Columns B-G
    # print(labels)
    
    # Check if all labels are present
    if not all(labels):
        raise ValueError("One or more labels in row 2 are missing.")

    # Initialize a dictionary to store the data for each label
    data = {label: [] for label in labels}
    
    # Get data from rows starting at B3 (columns B-G)
    for row in sheet.iter_rows(min_row=3, min_col=2, max_col=7, values_only=True):
        for col_index, value in enumerate(row):
            if value:  # Skip empty cells
                data[labels[col_index]].append(value)
    return data

In [58]:
# Example usage
file_path = './Photonic Companies.xlsx'  # Replace with your Excel file path
sheet_name = 'Startup'  # Replace with the name of the sheet

try:
    data = import_excel_data(file_path, sheet_name)
    print(data)
except Exception as e:
    print(f"Error: {e}")

{'Company Name': ['LightMatter', 'Lightelligence', 'Ayar Labs', 'PsiQuantum', 'Quantum Transistors', 'Q.ANT', 'Lumai ', 'iPRONICS', 'Salience Labs', 'Pixel Photonics', 'Dream Photonics ', 'Akhetonics', 'Celestial AI', 'Xscape Photonics', 'Optelligence', 'Nubis Communications', 'Voyant Photonics', 'Aegiq', 'Nu-quantum', 'Miraex', 'Nordic Quantum Computing Group', 'Orca Computing', 'Photonic Inc', 'Quandela ', 'Quix', 'Sparrow Quantum', 'Lumiphase', 'Enosemi', 'Nexus Photonics', 'Black Semiconductor'], 'Size': [200, 200, '200+', 280, 50, 100, 20, '< 50', '< 50', '< 50', 10, '< 50', '< 200', '< 50', '< 3', '< 50 ', '< 30', '< 50 ', '< 50 ', 50, 'CLOSED 11/2024', '~100', '< 50', '< 50', '< 50', '< 50 ', '< 100 ', '< 50 ', '< 20 ', '<200'], 'Focus': ['Optical MVM / PNN', 'Optical MVM / Networking', 'Optical I/O for AI, Interconnection ', 'Quantum Computing / Optical Interconnect', 'solid-state\xa0quantum\xa0processor', 'Optical MVM', '3D optical AI computing ', 'Microwave Photonics and Netw

NameError: name 'prin' is not defined